In [0]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'student-spending-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4544119%2F7768117%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240316%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240316T153110Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D3bc56e4c8cde67dd752b5fa2bd6326488b4424e47430afa06d2f9b6dfdf04e6d5f87fa219f3295ba87ce22e189d2807b547f5a5487b400359dab8ba35934ddbe4026bd3ed03dfcd9e956e7940f8c5a5eee1d30febde902ec4d6a83c1b64eb436bc32b7845dc67d79d0782d058293eea315fd9e4b42063e87e4eaade556a6a7e1111ef75e5624343ca2aaaddde8e1276e9ffe29cd4078d84e43eac9b5e9b37b3a2073fef7d645aea0a84b584af45006c9f812f5d27520b4d334504980b8053a8c020ca09daefb9d7b66272b9f2e983ae7e1cc24466e1fb71022b7cf547621571e93194d2e300bc33c2ffa32525adee52600796615e03ead770ce8da7ffa57c45a'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

### Disclaimer
* I am someone who is still fairly new to the field of data science and machine learning so much of my code and practices may not be ideal. But I do hope that this notebook can provide you with a good general understanding of using machine learning for regression.

* If you are more experienced than me and notice problems, please feel free to let me know.

* Most of the information in this notebook on building the model is from the course "Supervised Machine Learning Regression Classification" by Andrew Ng on Coursera. Please check out that course if you're interested, the explainations in that course were honestly amazing.

# Objective
This will be a simple practice lab of implementing multivariate regression in order to predict the spending habits of a given student.

The data given by the data set is purely fictional, therefore this notebook should only be used for practicing data analysis techniques and should not be used for any actual application.

# Exploratory Data Analysis
Here we will view and explore the data to get a better understanding of what features we can include in our regression.

In [ ]:
# viewing the data
spending = pd.read_csv('/kaggle/input/student-spending-dataset/student_spending (1).csv')

print('First Few rows')
print(spending.head())
print('\n')
print('Info')
print(spending.info())

In [ ]:
# looking at missing values
print(spending.isna().sum())

* Wowie no filling na needs to be done!! But we don't need this unnamed column since by the looks of it, it's just another index column and may get in the way of things later.

In [ ]:
spending.drop(labels = 'Unnamed: 0', axis = 1, inplace = True)

### Now lets break down the spending into graphs that are easier to interpret and then compare it across groups
How can we do this?
1. Create a new pandas series with the mean for all students, this will just essentially be our baseline
2. Then we create more series with the mean for students of a group that we are filtering by i.e. Major, Year, Gender, Age
3. Lastly we compare across groups

In [ ]:
# the actual things that students spend money on
spendings = ['tuition', 'housing', 'food', 'transportation', 'books_supplies', 'entertainment', 'personal_care', 'technology', 'health_wellness', 'miscellaneous']

In [ ]:
# creating series for spendings
total_mean = spending[spendings]
total_mean = total_mean.mean()

print(total_mean)

In [ ]:
categories = total_mean.index
heights = total_mean.values

bottom = 0
for category, height in zip(categories, heights):
    plt.bar('Total Spending', height, bottom=bottom, label=category)
    bottom += height
    
plt.ylabel('Spending')
plt.title('Stacked Bar Chart of Mean Spending')
plt.legend()

print('Mean Spending for a Student is ', total_mean.sum())

* We see that the vast majority of this spending is consumed by the tuition. Now lets view the spending seperated by the major of the individual.

In [ ]:
total_spend = total_mean.sum()
print(total_spend)

major_mean = spending[spending['major'] == 'Psychology']
major_mean = major_mean[spendings]
major_mean = major_mean.mean()
comp_sci_spend = major_mean.sum()
print(comp_sci_spend)

In [ ]:
majors = spending['major'].unique()
print(majors)

In [ ]:
# mean spending by major
# for ax, major in zip(axes.flatten(), majors):
#     major_mean = spending[spending['major'] == major]
#     # it's probably not best practice to have a variable named spending and another named spendings
#     major_mean = major_mean[spendings]
#     major_mean = major_mean.mean()
#     categories = major_mean.index
#     heights = major_mean.values

#     bottom = 0
#     for category, height in zip(categories, heights):
#         ax.bar('Total Spending', height, bottom=bottom, label=category)
#         bottom += height

#     ax.set_ylabel('Spending')
#     ax.set_title(f'Mean Spending for {major} Majors')
#     ax.legend()

# ^ code for plotting stacked bar charts, not working too well and also stacked bar charts not
# necessary as this is mostly for comparing the difference in total spending habits rather than
# by category


by_majors = spending.groupby('major')[spendings].mean().reset_index() #return a dataframe with the mean spending of each major

# making bar charts
category = by_majors['major'].values
heights = by_majors[spendings].sum(axis = 1).values

plt.bar(category, heights)
plt.ylabel('Spending')
plt.ylim(6000, 6500)


**Observations:**
There seems to be some difference in the spendings by majors but not much. This could be because the dataset used is fictional and thus does not take into account the different factors that can influence the spendings of a student.

But lets Try this again but with different features such as age, gender, monthly income, financial aid, and preferred payment method

In [ ]:
# spending by gender
genders = spending['gender'].unique()

by_genders = spending.groupby('gender')[spendings].mean().reset_index()

# making bar charts
category = by_genders['gender'].values
heights = by_genders[spendings].sum(axis = 1).values

plt.bar(category, heights)
plt.ylabel('Spending')
plt.ylim(6000,6500)

Again there is little to no difference

In [ ]:
# spending by preferred payment method
pref_pay_method = spending['preferred_payment_method'].unique()

by_pref_pay_method = spending.groupby('preferred_payment_method')[spendings].mean().reset_index()

# making bar charts
category = by_pref_pay_method['preferred_payment_method'].values
heights = by_pref_pay_method[spendings].sum(axis = 1).values

plt.bar(category, heights)
plt.ylabel('Spending')
plt.ylim(6000,6500)

Still very minor differences

In [ ]:
# spending by age
age = spending['age'].unique()

by_age = spending.groupby('age')[spendings].mean().reset_index()

# making bar charts
category = by_age['age'].values
heights = by_age[spendings].sum(axis = 1).values

plt.bar(category, heights, width = 0.8)
plt.ylabel('Spending')
plt.ylim(6000,6500)

Wowie there appears to be some sort of correlation between spending and age but keep in mind that this is a very zoomed in view of the graph.

In [ ]:
# spending by monthly_income
by_monthly_inc = spending['monthly_income'].reset_index()
by_spendings = spending[spendings].reset_index()
by_monthly_inc['spending'] = by_spendings.sum(axis = 1)

plt.scatter(by_monthly_inc['monthly_income'], by_monthly_inc['spending'])
plt.ylabel('Spending')
plt.xlabel('Monthly Income')

Little to no correlation between spending and monthly income

In [ ]:
# spending by financial_aid
by_fin_aid = spending['financial_aid'].reset_index()
by_spendings = spending[spendings].reset_index()
by_fin_aid['spending'] = by_spendings.sum(axis = 1)

plt.scatter(by_fin_aid['financial_aid'], by_fin_aid['spending'])
plt.ylabel('Spending')
plt.xlabel('Financial Aid')

Little Correlation between spending and financial aid

### Splitting Data into Training and Test Set
**Purpose of Each Set:**
Training: Meant to train the model and obtain optimized parameters
Testing: Taking the final model with the optimized parameters and testing it. Meant to be a simulation of a real life application of the data.

Since Our Data is relatively small (1000 observations) we will have relatively large cross validation and test sets with respect to the training set. So we will use 70% for testing and 30% for testing. These numbers are fairly arbitrary so they can be modified depending on preferences.

In [ ]:
# should have done this a lot earlier to make the visualization simpler ehe
# don't be like me, do this when cleaning your data
spending['Spending'] = spending[spendings].sum(axis = 1)
print(spending.head())

The Regression Model will take in the features of
* age
* gender
* major
* payment method

As these are the only features that had some sort of correlation with the total spending of the student. Try playing around with the features and seeing how the model does.

In [ ]:
features = ['age', 'gender', 'major', 'preferred_payment_method']
# we also have to clean up the spendings dataframe a bit since some of the features are categorical
# turn categorical variables into dummy variables

In [ ]:
for feature in features[1:]:
    print(f'Unique number of categories in {feature}: {spending[feature].unique()}')

In [ ]:
dummy_df1 = pd.get_dummies(spending['gender'])
dummy_df2 = pd.get_dummies(spending['major'])
dummy_df3 = pd.get_dummies(spending['preferred_payment_method'])

dummy_df = pd.concat([dummy_df1, dummy_df2, dummy_df3], axis=1)
dummy_df['age'] = spending['age']
print(dummy_df.head())

In [ ]:
# splitting data
X_train = dummy_df.iloc[:700].values
Y_train = spending.iloc[:700]['Spending'].values

X_test = dummy_df.iloc[700:1000].values
Y_test = spending.iloc[700:1000]['Spending'].values

print(X_train.shape)
print(Y_train.shape)

# Building the Regression Model

\*Note: the following code takes a lot of code and equations from the labs provided in the Machine Learning Specialization by Andrew Ng on Coursera.

## How Linear Regression Works

1. General Equation for the predicted value in univariate linear regression. Multivariate regression essentially works in the same way but with more weightings and more variables x.
    $$f_{w,b}(x) = wx + b$$
    

2. Compute the cost. For now we are just going to use mean square error because it is simple and it gets the job done.
    - Cost Function:

    $$J(\mathbf{w},b) = \frac{1}{2m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})^2 \tag{3}$$ 
where:
$$ f_{\mathbf{w},b}(\mathbf{x}^{(i)}) = \mathbf{w} \cdot \mathbf{x}^{(i)} + b  \tag{4} $$ 

3. Use Gradient Descent to find the optimal parameters for w and b given your dataset
      - Gradient Descent:
      $$b := b -  \alpha \frac{\partial J(w,b)}{\partial b}$$
      $$w := w -  \alpha \frac{\partial J(w,b)}{\partial w}$$
      Where $\alpha$ is the learning rate and $\frac{\partial J(w,b)}{\partial variable}$ is the partial derivative of the cost function with respect to one of the parameters
      
\* more will be explained about Cost and Gradient Descent Later

### Cost Function

The cost function can be thought of as a measure of how close the prediction made by the model given the parameters **w** and **b** are to the actual output. As **w** and **b** change with gradient descent, the loss associated with the parameters for **w** and **b** will also change.

In [ ]:
def compute_cost(X, Y, w, b):
    """
    X: numpy array: Shape (m,n)   Input Data
    Y: numpy array: Shape (m,)    Expected Output
    w: numpy array: Shape (n,)    Weightings
    b: scalar:  Bias
    """
    
    m = X.shape[0]
    cost = 0.0
    for i in range(m):                                
        f_wb_i = np.dot(X[i], w) + b
        cost = cost + (f_wb_i - Y[i])**2       
    cost = cost / (2 * m)                      
    return cost

# the code below provides a more detailed idea of what is going on but runs a lot slower since it does not utilize vectorization with np.dot()

# m, n = X.shape
#     total_cost = 0
#     cost_sum = 0
#     for i in range(m):
#         f_wb = 0
#         for j in range(n):
#             f_wb_j = w[j] * X[i][j]
#             f_wb += f_wb_j
#         f_wb += b
#         cost_i = (f_wb - y[i])**2
#         cost_sum = cost_sum + cost_i
        
#     total_cost = cost_sum / (2*m)

### Gradient Descent
The general formula for gradient descent in univariate linear regression is:
$$b := b -  \alpha \frac{\partial J(w,b)}{\partial b}$$
$$w := w -  \alpha \frac{\partial J(w,b)}{\partial w}$$

where, parameters **w** and **b** are both updated simultaniously and where  
$$
\begin{align}
\frac{\partial J(\mathbf{w},b)}{\partial w_j}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})x_{j}^{(i)} \tag{6}  \\
\frac{\partial J(\mathbf{w},b)}{\partial b}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)}) \tag{7}
\end{align}
$$


In [ ]:
def compute_gradient(X, Y, w, b): 
    """
    X: numpy array: Shape (m,n)   Input Data
    Y: numpy array: Shape (m,)    Expected Output
    w: numpy array: Shape (n,)    Weightings
    b: scalar:  Bias
     """
    m,n = X.shape
    dj_dw = np.zeros((n,))
    dj_db = 0.

    for i in range(m):                             
        err = (np.dot(X[i], w) + b) - Y[i]   
        for j in range(n):                         
            dj_dw[j] = dj_dw[j] + err * X[i, j]    
        dj_db = dj_db + err                        
    dj_dw = dj_dw / m                                
    dj_db = dj_db / m                                
    
    return dj_db, dj_dw

In [ ]:
import math
def gradient_descent(X, Y, w, b, cost_function, gradient_function, alpha, num_iters): 
    """
    X: numpy array: Shape (m,n)   Input Data
    y: numpy array: Shape (m,)    Expected Output
    w: numpy array: Shape (n,)    Weightings
    b: float                      Bias
    cost_function:                function to compute cost made earlier
    gradient_function:            function to compute the gradient
    alpha: (float):               Learning rate
    num_iters: (int):             number of iterations to run gradient descent
      """    
    for i in range(num_iters):

        # Computing the gradient 
        dj_db,dj_dw = gradient_function(X, Y, w, b)

        # Update Parameters using w, b, alpha and gradient
        w = w - alpha * dj_dw  
        b = b - alpha * dj_db 
        
        # Print cost every at intervals 10 times or as many iterations if < 10
        if i% math.ceil(num_iters / 10) == 0:
            J = cost_function(X, Y, w, b)
            print(f"Iteration {i:4d}: Cost {J:8.2f}   ")
        
    return w, b # returning the optimal w and b parameters found using the model

# Testing
The fun stuff!!!!

In [ ]:
print(X_train)

In [ ]:
# training the model
w = np.random.randn(12,) # the reason it is 12 is because there are 12 features in x train
b = np.random.randn()
alpha = 0.003 # found using trial and error
num_iters = 30000

w_opt, b_opt = gradient_descent(X_train, Y_train, w, b, compute_cost, compute_gradient, alpha, num_iters)

In [ ]:
# testing the model:
m = X_test.shape[0]
for i in range(10):
    rand_index = np.random.randint(m)
    prediction = np.dot(X_test[i], w_opt) + b_opt
    actual = Y_test[i]
    print(f"prediction: {prediction:0.2f}, target value: {actual}, discrepency: {prediction - actual:0.2f}")

### Conclusions
Ok so it seems that our model isn't really able to predict the actual value of the spending, this can be potentially due to many reason:
- Overfitting: By running too many iterations, our model may be memorizing the training data rather than being able to generalize from the training data to make predictions.
- Inadequate features: perhaps there were other features that we could have trained the model on but didn't due to not going in depth enough during the exploratory phase of data analysis
- Nature of the data: given that the data is fictional, there could be a possibility that there is little correlation between the features and the actual data.